### Generating Random Input

In [87]:
import random
from datetime import datetime, timedelta

In [88]:
CENTER_LAT, CENTER_LNG = 18.5204, 73.8567

In [89]:
def random_location():
    return (
        CENTER_LAT + random.uniform(-0.02, 0.02),
        CENTER_LNG + random.uniform(-0.02, 0.02)
    )

In [90]:
def generate_riders(num_riders=100):
    riders = []
    for i in range(num_riders):
        lat, lng = random_location()
        status = random.choice(["idle", "on_gig"])
        km_to_finish = round(random.uniform(0.5, 5.0), 2) if status == "on_gig" else 0.0
        est_finish_ts = (datetime.utcnow() + timedelta(minutes=int(km_to_finish * 2))).isoformat() + 'Z'
        riders.append({
            "rider_id": f"R{i:03}",
            "lat": lat,
            "lng": lng,
            "soc_pct": random.randint(15, 80),
            "status": status,
            "km_to_finish": km_to_finish,
            "est_finish_ts": est_finish_ts if status == "on_gig" else None
        })
    return riders

In [91]:
def generate_stations():
    stations = []
    for i in range(3):
        lat, lng = random_location()
        stations.append({
            "station_id": f"S_{chr(65+i)}",
            "lat": lat,
            "lng": lng,
            "queue_len": random.randint(0, 3)
        })
    return stations

### Utility Functions

In [92]:
import math
import json
from datetime import datetime, timedelta

In [93]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = math.sin(d_lat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [94]:
def add_minutes(iso_ts, minutes):
    dt = datetime.fromisoformat(iso_ts.replace('Z', ''))
    new_dt = dt + timedelta(minutes=minutes)
    return new_dt.isoformat() + 'Z'

In [95]:
def now_iso():
    return datetime.utcnow().isoformat() + 'Z'

In [96]:
def save_plan_output(plan):
    with open("plan_output.json", "w") as f:
        json.dump(plan, f, indent=2)

### Optimizer

In [97]:
import heapq
from datetime import datetime, timedelta

In [98]:
BATTERY_CONSUMPTION_PER_KM = 4
SWAP_DURATION_MIN = 4
MIN_SOC_ALLOWED = 10

In [99]:

def run_optimizer(riders, stations):
    plan = []
    station_queues = {s['station_id']: [] for s in stations}
    current_ts = now_iso()

    for station in stations:
        for _ in range(station['queue_len']):
            station_queues[station['station_id']].append(datetime.fromisoformat(current_ts.replace("Z", "")))

    for rider in riders:
        current_lat, current_lng = rider["lat"], rider["lng"]
        status = rider["status"]
        soc = rider["soc_pct"]

        if status == "on_gig":
            km_to_finish = rider["km_to_finish"]
            soc -= km_to_finish * BATTERY_CONSUMPTION_PER_KM
            if soc < 0:
                soc = 0
            current_lat += 0.001
            current_lng += 0.001
            depart_ts = rider["est_finish_ts"]
        else:
            depart_ts = current_ts

        if soc < MIN_SOC_ALLOWED + 4 * 1.5:
            best_station = None
            min_dist = float("inf")

            for station in stations:
                dist = haversine(current_lat, current_lng, station["lat"], station["lng"])
                if soc - (dist * BATTERY_CONSUMPTION_PER_KM) >= MIN_SOC_ALLOWED and dist < min_dist:
                    min_dist = dist
                    best_station = station

            if best_station:
                travel_time = int((min_dist / 30) * 60)
                arrive_ts = add_minutes(depart_ts, travel_time)

                swap_queue = station_queues[best_station['station_id']]
                swap_start = datetime.fromisoformat(arrive_ts.replace("Z", ""))
                while len([t for t in swap_queue if abs((swap_start - t).seconds / 60) < SWAP_DURATION_MIN]) >= 5:
                    swap_start += timedelta(minutes=1)
                swap_end = swap_start + timedelta(minutes=SWAP_DURATION_MIN)

                swap_queue.append(swap_start)

                plan.append({
                    "rider_id": rider["rider_id"],
                    "station_id": best_station["station_id"],
                    "depart_ts": depart_ts,
                    "arrive_ts": arrive_ts,
                    "swap_start_ts": swap_start.isoformat() + "Z",
                    "swap_end_ts": swap_end.isoformat() + "Z",
                    "eta_back_lat": best_station["lat"],
                    "eta_back_lng": best_station["lng"]
                })

    return plan

### Generating Riders

In [100]:
import pandas as pd

In [101]:
riders = generate_riders(num_riders=100)

C:\Users\prera\AppData\Local\Temp\ipykernel_16720\2583519428.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  est_finish_ts = (datetime.utcnow() + timedelta(minutes=int(km_to_finish * 2))).isoformat() + 'Z'


In [102]:
rider_df = pd.DataFrame(riders)
rider_df.head()

,rider_id,lat,lng,soc_pct,status,km_to_finish,est_finish_ts
0,R000,18.507328,73.840825,37,on_gig,1.47,2025-05-21T11:32:03.278121Z
1,R001,18.506820,73.876542,48,idle,0.00,None
2,R002,18.515020,73.847838,43,on_gig,4.09,2025-05-21T11:38:03.278180Z
3,R003,18.527975,73.860868,33,on_gig,2.85,2025-05-21T11:35:03.278190Z
4,R004,18.522392,73.863711,21,idle,0.00,None


### Generating Stations

In [103]:
stations = generate_stations()

In [104]:
station_df = pd.DataFrame(stations)
station_df.head()

,station_id,lat,lng,queue_len
0,S_A,18.514667,73.873062,1
1,S_B,18.515197,73.843873,2
2,S_C,18.525561,73.853671,2


### Running optimizer

In [105]:
plan_output = run_optimizer(riders, stations)

C:\Users\prera\AppData\Local\Temp\ipykernel_16720\87406163.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().isoformat() + 'Z'


### Optimized Plan

In [106]:
output_plan = pd.DataFrame(plan_output)
output_plan.head()

,rider_id,station_id,depart_ts,arrive_ts,swap_start_ts,swap_end_ts,eta_back_lat,eta_back_lng
0,R022,S_C,2025-05-21T11:31:03.278332Z,2025-05-21T11:31:03.278332Z,2025-05-21T11:31:03.278332Z,2025-05-21T11:35:03.278332Z,18.525561,73.853671
1,R086,S_A,2025-05-21T11:37:03.278789Z,2025-05-21T11:37:03.278789Z,2025-05-21T11:37:03.278789Z,2025-05-21T11:41:03.278789Z,18.514667,73.873062


### Saving the output plan in .json format

In [107]:
save_plan_output(plan_output)